# TS-Fresh

Making Model

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import datetime
import numpy as np
import sklearn
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
weekend = ['Saturday','Sunday']
def week_day_type(x):
    if x in weekend:
        return 'weekends'
    else:
        return 'weekdays'
def time_slot(x):
    if x in morning:
        return 'morning'
    elif x in afternoon:
        return 'afternoon'
    elif x in evening:
        return 'evening'
    else:
        return 'night'  
df = pd.read_csv("../Part_2/energydata_complete.csv")
df['date']=pd.to_datetime(df['date'])
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df['day_of_week']=df['date'].dt.weekday_name
df['time_hr_24']=df['date'].dt.hour
df['time_min']=df['date'].dt.minute
df['week_day_type']=df['day_of_week'].map(week_day_type)
morning=range(6,12)
afternoon=range(12,17)
evening=range(17,22)  
df['time_slot']=df['time_hr_24'].map(time_slot)
#df.drop(['date'],axis=1,inplace=True)
df=pd.get_dummies(df,prefix=['DOW','TS','WDT'],columns=['day_of_week','time_slot','week_day_type'])
#Removing Out-liers
print(df.shape)
df=df[(df['Appliances']-df['Appliances'].mean()).abs() <= 3*df['Appliances'].std()]
df=df[(df['lights']-df['lights'].mean()).abs() <= 3*df['lights'].std()]
df=df[(df['T1']-df['T1'].mean()).abs() <= 3*df['T1'].std()]
df=df[(df['T2']-df['T2'].mean()).abs() <= 3*df['T2'].std()]
df=df[(df['T3']-df['T3'].mean()).abs() <= 3*df['T3'].std()]
df=df[(df['T4']-df['T4'].mean()).abs() <= 3*df['T4'].std()]
df=df[(df['T5']-df['T5'].mean()).abs() <= 3*df['T5'].std()]
df=df[(df['T6']-df['T6'].mean()).abs() <= 3*df['T6'].std()]
df=df[(df['T7']-df['T7'].mean()).abs() <= 3*df['T7'].std()]
df=df[(df['T8']-df['T8'].mean()).abs() <= 3*df['T8'].std()]
df=df[(df['T9']-df['T9'].mean()).abs() <= 3*df['T9'].std()]
df=df[(df['T_out']-df['T_out'].mean()).abs() <= 3*df['T_out'].std()]
df=df[(df['RH_1']-df['RH_1'].mean()).abs() <= 3*df['RH_1'].std()]
df=df[(df['RH_2']-df['RH_2'].mean()).abs() <= 3*df['RH_2'].std()]
df=df[(df['RH_3']-df['RH_3'].mean()).abs() <= 3*df['RH_3'].std()]
df=df[(df['RH_4']-df['RH_4'].mean()).abs() <= 3*df['RH_4'].std()]
df=df[(df['RH_5']-df['RH_5'].mean()).abs() <= 3*df['RH_5'].std()]
df=df[(df['RH_6']-df['RH_6'].mean()).abs() <= 3*df['RH_6'].std()]
df=df[(df['RH_7']-df['RH_7'].mean()).abs() <= 3*df['RH_7'].std()]
df=df[(df['RH_8']-df['RH_8'].mean()).abs() <= 3*df['RH_8'].std()]
df=df[(df['RH_9']-df['RH_9'].mean()).abs() <= 3*df['RH_9'].std()]
df=df[(df['RH_out']-df['RH_out'].mean()).abs() <= 3*df['RH_out'].std()]
df=df[(df['Press_mm_hg']-df['Press_mm_hg'].mean()).abs() <= 3*df['Press_mm_hg'].std()]
df=df[(df['Windspeed']-df['Windspeed'].mean()).abs() <= 3*df['Windspeed'].std()]
df=df[(df['Visibility']-df['Visibility'].mean()).abs() <= 3*df['Visibility'].std()]
df=df[(df['Tdewpoint']-df['Tdewpoint'].mean()).abs() <= 3*df['Tdewpoint'].std()]
df=df[(df['rv1']-df['rv1'].mean()).abs() <= 3*df['rv1'].std()]
df=df[(df['rv2']-df['rv2'].mean()).abs() <= 3*df['rv2'].std()]
print(df.shape)

(19735, 47)
(17118, 47)


In [4]:
x = df.drop(['Appliances'],axis=1)
#x.drop(['Appliances'],axis=1,inplace=True)
x.head()

,date,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday,TS_afternoon,TS_evening,TS_morning,TS_night,WDT_weekdays,WDT_weekends
27,2016-01-11 21:30:00,20,21.356667,45.826667,20.666667,45.163333,20.390000,46.090000,19.390000,47.500000,...,0,0,0,0,0,1,0,0,1,0
28,2016-01-11 21:40:00,20,21.390000,45.690000,20.700000,45.060000,20.390000,46.090000,19.426667,47.993333,...,0,0,0,0,0,1,0,0,1,0
29,2016-01-11 21:50:00,20,21.500000,45.333333,20.700000,44.933333,20.390000,46.060000,19.566667,48.466667,...,0,0,0,0,0,1,0,0,1,0
30,2016-01-11 22:00:00,20,21.500000,45.126667,20.790000,44.633333,20.390000,46.000000,19.666667,48.093333,...,0,0,0,0,0,0,0,1,1,0
31,2016-01-11 22:10:00,20,21.533333,44.966667,20.790000,44.360000,20.426667,45.933333,19.600000,47.500000,...,0,0,0,0,0,0,0,1,1,0


In [5]:
x = pd.Series(data=df['Appliances'].values, index=df['date'])

Checking the content of the data.

In [6]:
x

date
2016-01-11 21:30:00    100
2016-01-11 21:40:00    100
2016-01-11 21:50:00    100
2016-01-11 22:00:00    110
2016-01-11 22:10:00    400
2016-01-11 22:20:00    400
2016-01-11 22:40:00    240
2016-01-11 22:50:00     60
2016-01-11 23:00:00     60
2016-01-11 23:10:00     60
2016-01-11 23:20:00     50
2016-01-11 23:30:00     70
2016-01-11 23:40:00     60
2016-01-11 23:50:00     40
2016-01-12 00:00:00     40
2016-01-12 00:10:00     30
2016-01-12 00:20:00     40
2016-01-12 00:30:00     50
2016-01-12 00:40:00    310
2016-01-12 00:50:00    380
2016-01-12 01:00:00    380
2016-01-12 01:10:00    370
2016-01-12 01:20:00    120
2016-01-12 01:30:00     50
2016-01-12 01:40:00     40
2016-01-12 01:50:00     50
2016-01-12 02:00:00     40
2016-01-12 02:10:00     50
2016-01-12 02:20:00     50
2016-01-12 02:30:00     50
                      ... 
2016-05-27 06:30:00     50
2016-05-27 06:40:00     50
2016-05-27 06:50:00     50
2016-05-27 07:00:00     50
2016-05-27 07:10:00     60
2016-05-27 07:20:00    

In [7]:
df1 = pd.DataFrame(x)
df1.reset_index(inplace=True)
# df.columns = ["time", "value"]
# df["kind"] = "a"
# df["id"] = 1

In [8]:
df1.head()

,date,0
0,2016-01-11 21:30:00,100
1,2016-01-11 21:40:00,100
2,2016-01-11 21:50:00,100
3,2016-01-11 22:00:00,110
4,2016-01-11 22:10:00,400


Performing tsfresh

In [9]:
from tsfresh.utilities.dataframe_functions import roll_time_series

In [10]:
df_shift, y = make_forecasting_frame(x, kind="price", max_timeshift=10, rolling_direction=1)
df_shift

,time,value,id,kind
154008,2016-01-11 21:30:00,100.0,2016-01-11 21:40:00,price
136892,2016-01-11 21:30:00,100.0,2016-01-11 21:50:00,price
154009,2016-01-11 21:40:00,100.0,2016-01-11 21:50:00,price
119777,2016-01-11 21:30:00,100.0,2016-01-11 22:00:00,price
136893,2016-01-11 21:40:00,100.0,2016-01-11 22:00:00,price
154010,2016-01-11 21:50:00,100.0,2016-01-11 22:00:00,price
102663,2016-01-11 21:30:00,100.0,2016-01-11 22:10:00,price
119778,2016-01-11 21:40:00,100.0,2016-01-11 22:10:00,price
136894,2016-01-11 21:50:00,100.0,2016-01-11 22:10:00,price
154011,2016-01-11 22:00:00,110.0,2016-01-11 22:10:00,price


In [11]:
X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute, 
                     show_warnings=False)

Feature Extraction: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [31:14<00:00, 174.23s/it]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_tre

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


In [12]:
y.head()

date
2016-01-11 21:40:00    100
2016-01-11 21:50:00    100
2016-01-11 22:00:00    110
2016-01-11 22:10:00    400
2016-01-11 22:20:00    400
Name: value, dtype: int64

In [13]:
from tsfresh import extract_relevant_features

Extracting relevant features.

In [14]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(X)
features_filtered = select_features(X, y)

In [15]:
features_filtered.head()

variable,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5",value__quantile__q_0.3,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5",value__quantile__q_0.4,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20",value__median,value__quantile__q_0.2,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_5","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_10",...,"value__fft_coefficient__coeff_3__attr_""imag""",value__partial_autocorrelation__lag_6,value__number_cwt_peaks__n_5,value__energy_ratio_by_chunks__num_segments_10__segment_focus_7,"value__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.4","value__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2","value__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2","value__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.6",value__autocorrelation__lag_5,value__approximate_entropy__m_2__r_0.7
id,,,,,,,,,,,,,,,,,,,,,
2016-01-11 21:40:00,91.421879,100.0,111.541685,100.0,87.114634,66.239412,100.0,100.0,127.137725,103.020748,...,-0.756345,-0.593199,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,0.000000
2016-01-11 21:50:00,91.421879,100.0,111.541685,100.0,87.114634,66.239412,100.0,100.0,127.137725,103.020748,...,-0.756345,-0.593199,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,0.000000
2016-01-11 22:00:00,91.421879,100.0,111.541685,100.0,87.114634,66.239412,100.0,100.0,127.137725,103.020748,...,-0.756345,-0.593199,1.0,0.0,0.0,0.0,0.0,0.0,-0.333333,0.000000
2016-01-11 22:10:00,91.421879,100.0,111.541685,100.0,87.114634,66.239412,100.0,100.0,127.137725,103.020748,...,-0.756345,-0.593199,1.0,0.0,0.0,0.0,0.0,0.0,-0.333333,0.056633
2016-01-11 22:20:00,91.421879,100.0,111.541685,100.0,87.114634,66.239412,100.0,100.0,127.137725,103.020748,...,-0.756345,-0.593199,1.0,0.0,0.0,0.0,0.0,0.0,-0.333333,0.074179


In [16]:
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import numpy as np

Extracting final output.

In [17]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(features_filtered,y,test_size=0.3,random_state=42)

# model = RandomForestRegressor(n_estimators=150, max_features='sqrt', n_jobs=-1)  # случайный лес
models = [LinearRegression(),
          RandomForestRegressor(),
          MLPRegressor(hidden_layer_sizes=(155))
          ]

TestModels = pd.DataFrame()
tmp = {}

for model in models:
    # get model name
    m = str(model)
    tmp['Model'] = m[:m.index('(')]
    # fit model on training dataset
    model.fit(Xtrn, Ytrn)
    # predict consumption
    predictions = model.predict(Xtest)
    #Evaluation for Testing set
    #R2 score
    tmp['R2_Test'] = r2_score(Ytest,predictions)
    #Mean Absolute Error(MAE)
    tmp['MAE_Test']= mean_absolute_error(Ytest,predictions)
    #Mean Squared Error(MSE)
    tmp['MSE_Test']= mean_squared_error(Ytest,predictions)
    #Root Mean Squared Error (RMSE)
    tmp['RMSE_Test'] = np.sqrt(mean_squared_error(Ytest,predictions))
    #Evaluation for Training test
    predictions_trn = model.predict(Xtrn)
    #R2_Score
    tmp['R2_Train'] = r2_score(Ytrn,predictions_trn)
    #Mean Absolute Error(MAE)
    tmp['MAE_Train']= mean_absolute_error(Ytrn,predictions_trn)
    #Mean Squared Error(MSE)
    tmp['MSE_Train']= mean_squared_error(Ytrn,predictions_trn)
    #Root Mean Squared Error (RMSE)
    tmp['RMSE_Train'] = np.sqrt(mean_squared_error(Ytrn,predictions_trn))

    # write obtained data
    TestModels = TestModels.append([tmp])

TestModels.set_index('Model', inplace=True)

print(TestModels)

                           MAE_Test     MAE_Train      MSE_Test     MSE_Train  \
Model                                                                           
LinearRegression       2.622187e+01  2.409460e+01  6.085721e+03  2.077187e+03   
RandomForestRegressor  2.465078e+01  9.751281e+00  2.382641e+03  4.056849e+02   
MLPRegressor           1.188270e+08  3.977830e+08  3.407596e+19  6.007033e+20   

                            R2_Test      R2_Train     RMSE_Test    RMSE_Train  
Model                                                                          
LinearRegression      -4.474209e-01  5.351238e-01  7.801103e+01  4.557617e+01  
RandomForestRegressor  4.333154e-01  9.092074e-01  4.881230e+01  2.014162e+01  
MLPRegressor          -8.104589e+15 -1.344379e+17  5.837462e+09  2.450925e+10  
